# NIST PREP Summer 2020

- This is a summary of my accomplishments in the NIST PREP program
- Data was obtained from the Windover Group, NIST Gaithersburg

- loading standard Python packages and enabling plotting

In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from matplotlib.patches import Ellipse
%matplotlib inline

- include custom gaussian for simple gaussian fits
- include custom bruker_io toolbox
- include custom Spectrum_Evaulation toolbox


In [ ]:
import gaussian
import bruker_io as brio
import spectrum_evaluation as se
